# 👗 DemandAI - H&M Fashion Training Notebook (Enterprise + Transformer + Max Training)
This notebook trains your AI on **H&M Personalized Fashion Data**.

### **Instructions**
1.  **Menu**: Runtime > Change runtime type > Select **T4 GPU** (Faster training).
2.  **Run Step 1**: Install Dependencies.
3.  **Run Step 2**: Generate High-Fidelity H&M Synthetic Data (200,000 Rows).
4.  **Run Step 3**: Train the **Transformer** Deep Learning Model (50 Epochs).
5.  **Run Step 4**: Download your new `model.h5`.

In [ ]:
# [STEP 1] Install AI Libraries
!pip install tensorflow pandas scikit-learn numpy matplotlib faker
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
import matplotlib.pyplot as plt
from faker import Faker
import random

print("✅ Libraries Installed Successfully!")

In [ ]:
# [STEP 2] Generate H&M Style Dataset
# Enterprise Scale: 200,000 Rows to simulate 'Real Data' load.

print("🧵 Generating H&M Fashion Dataset (200k Transactions)...")
fake = Faker()
data = []
categories = ['Ladieswear', 'Divided', 'Menswear', 'Baby/Children', 'Sport']
product_types = ['Trousers', 'Dress', 'Sweater', 'T-shirt', 'Jacket']

for _ in range(200000):  # 200,000 Transactions
    date = fake.date_between(start_date='-2y', end_date='today') # Increased history to 2 years
    cat = random.choice(categories)
    ptype = random.choice(product_types)
    price = round(random.uniform(10.99, 99.99), 2)
    
    # Demand Logic: Winter sells more Sweaters, Summer sells T-shirts
    base_vol = random.randint(10, 50)
    season_mult = 1.0
    month = date.month
    
    if ptype == 'Sweater' and month in [11, 12, 1, 2]: season_mult = 2.5
    if ptype == 'T-shirt' and month in [6, 7, 8]: season_mult = 2.0
    
    sales = int(base_vol * season_mult)
    
    data.append([date, cat, ptype, price, sales])

df = pd.DataFrame(data, columns=['t_dat', 'index_group_name', 'product_type_name', 'price', 'sales_channel_id'])
df['sales'] = df['sales_channel_id'] # remap for demo
df = df.sort_values('t_dat')
print(f"✅ Generated {len(df)} H&M Transactions.")
df.head()

In [ ]:
# [STEP 3] Train Valid Transformer Model

# 1. Preprocess
scaler = MinMaxScaler()
data_seq = df['sales'].values.reshape(-1, 1)
scaled_data = scaler.fit_transform(data_seq)

# 2. Create Sequences
X_train, y_train = [], []
lookback = 30

for i in range(lookback, len(scaled_data)):
    X_train.append(scaled_data[i-lookback:i, 0])
    y_train.append(scaled_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# 3. Build TRANSFORMER Model (Attention Mechanism)
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res

inputs = Input(shape=(lookback, 1))
x = transformer_encoder(inputs, head_size=256, num_heads=4, ff_dim=4, dropout=0.25)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(1)(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

# 4. Train
print("🚀 Training TRANSFORMER Brain on H&M Data (50 Epochs)...")
history = model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1)
print("✅ Training Complete (Transformer Architecture)!")

# Plot Loss
plt.plot(history.history['loss'])
plt.title('Transformer Training Loss')
plt.show()

In [ ]:
# [STEP 4] Download Trained Brain
model.save('model.h5')
from google.colab import files
files.download('model.h5')
print("✅ 'model.h5' downloaded! Upload this to your Dashboard.")